In [1]:
# Import packages
from osgeo import gdal
import os
import numpy as np
import rasterio
import re

NDVI

In [10]:
# AGGREGATION PER SEASON

os.chdir('C:/Users/mieke/Documents/Msc Thesis/random_forest/preprocess/NDVI/season') # Set path
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/Copernicus/NDVI/NDVI_2014_2019/Karnataka/aoi_rescaled/' # Set path
file_names = os.listdir(path) # contains all filenames within the selected folder
month_combis = [['02','03','04','05','06'], ['06','07','08','09','10'], ['10', '11', '12', '01', '02']]
seasons = ['summer', 'kharif', 'rabi']

for year in range(2016,2019):
    for m in month_combis:
        if m == month_combis[-1]:
            temp1 = [str(year) + x for x in m[:-2]]
            temp2 = [str(year+1) + x for x in m[3:]]
            year_month = temp1 + temp2
        else:
            year_month = [str(year) + x for x in m]

        group = []
        for k in range(len(m)):
            group = group + list(filter(lambda x: year_month[k] in x, file_names))

        n = len(group) # number of images within folder
        if n != 0: 
            images = []
            val_array = []
            bin_array = []

            for j in range(n):
                images.append(gdal.Open(path + group[j])) # Open each of the n images
                val_array.append(images[j].GetRasterBand(1).ReadAsArray().flatten())  # Read each of the n images as an array
                
            col = images[0].RasterXSize # number of columns
            rows = images[0].RasterYSize # number of rows
            driver = images[0].GetDriver()
            nodata_value = images[0].GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
            
            for k in range(n):
                bin_array.append(np.where(val_array[k] == nodata_value, 1, 0)) # For each image, create binary array where value is 1 if nodata pixel, 0 otherwise

            sum_counts = sum(bin_array) # For each pixel, we count the number of nodata values
            numbers = n - sum_counts # number of times we have data values per pixel
            no_data = np.where(sum_counts == n, 1, 0) # 1 if no data available for particular pixel, 0 otherwise

            # If numbers is 0 for a particular pixel, then we do not have any data on that pixel
            numbers = np.where(numbers == 0, 1, numbers) # Set to 1 as we can not divide by 0

            val_sum = sum(val_array) # Sum all values
            avg = (sum(val_array) - sum_counts * nodata_value + no_data * nodata_value) / numbers # Calculate the average-data
            avgdataMatrix= avg.reshape(rows,col)

            # Create a new raster to save the average-values
            raster_avg = driver.Create('NDVI_' + str(year) + '_' + seasons[month_combis.index(m)] + ".tiff", col, rows, 1, gdal.GDT_Float32)

            # Copy the properties of the original raster
            raster_avg.SetGeoTransform(images[0].GetGeoTransform())
            raster_avg.SetProjection(images[0].GetProjection())

            # Add the average values to newly created raster
            raster_avg.GetRasterBand(1).WriteArray(avgdataMatrix)
            raster_avg.GetRasterBand(1).SetNoDataValue(nodata_value)

            #close the raster in order yo open in Qgis o Arcgis without problems.
            raster_avg = None
            del raster_avg

In [3]:
# AGGREGATION PER SOWING/HARVEST PERIOD

os.chdir('C:/Users/mieke/Documents/Msc Thesis/random_forest/preprocess/NDVI/sow_harvest') # Set path
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/Copernicus/NDVI/NDVI_2014_2019/Karnataka/aoi_rescaled/' # Set path
file_names = os.listdir(path) # contains all filenames within the selected folder
month_combis = [['01','02'], ['02','03'], ['05', '06'], ['06', '07'], ['09', '10'], ['10', '11'], ['04'], ['08'], ['12']]
periods = ['rabi_harvest', 'summer_sow', 'summer_harvest', 'kharif_sow', 'kharif_harvest', 'rabi_sow', 'summer_between', 'kharif_between', 'rabi_between']

for year in range(2016,2019):
    for m in month_combis:
        if m == month_combis[0]:
            year = year + 1

        year_month = [str(year) + x for x in m]

        group = []
        for k in range(len(m)):
            group = group + list(filter(lambda x: year_month[k] in x, file_names))

        n = len(group) # number of images within folder
        if n != 0: 
            images = []
            val_array = []
            bin_array = []

            for j in range(n):
                images.append(gdal.Open(path + group[j])) # Open each of the n images
                val_array.append(images[j].GetRasterBand(1).ReadAsArray().flatten())  # Read each of the n images as an array
                
            col = images[0].RasterXSize # number of columns
            rows = images[0].RasterYSize # number of rows
            driver = images[0].GetDriver()
            nodata_value = images[0].GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
            
            for k in range(n):
                bin_array.append(np.where(val_array[k] == nodata_value, 1, 0)) # For each image, create binary array where value is 1 if nodata pixel, 0 otherwise

            sum_counts = sum(bin_array) # For each pixel, we count the number of nodata values
            numbers = n - sum_counts # number of times we have data values per pixel
            no_data = np.where(sum_counts == n, 1, 0) # 1 if no data available for particular pixel, 0 otherwise

            # If numbers is 0 for a particular pixel, then we do not have any data on that pixel
            numbers = np.where(numbers == 0, 1, numbers) # Set to 1 as we can not divide by 0

            val_sum = sum(val_array) # Sum all values
            avg = (sum(val_array) - sum_counts * nodata_value + no_data * nodata_value) / numbers # Calculate the average-data
            avgdataMatrix= avg.reshape(rows,col)

            if m == month_combis[0]:
                year = year - 1

            # Create a new raster to save the average-values
            raster_avg = driver.Create('NDVI_' + str(year) + '_' + periods[month_combis.index(m)] + ".tiff", col, rows, 1, gdal.GDT_Float32)

            # Copy the properties of the original raster
            raster_avg.SetGeoTransform(images[0].GetGeoTransform())
            raster_avg.SetProjection(images[0].GetProjection())

            # Add the average values to newly created raster
            raster_avg.GetRasterBand(1).WriteArray(avgdataMatrix)
            raster_avg.GetRasterBand(1).SetNoDataValue(nodata_value)

            #close the raster in order yo open in Qgis o Arcgis without problems.
            raster_avg = None
            del raster_avg

LAI

In [4]:
# AGGREGATION PER SEASON

os.chdir('C:/Users/mieke/Documents/Msc Thesis/random_forest/preprocess/LAI/season') # Set path
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/Copernicus/LAI/Karnataka/aoi_rescaled/' # Set path
file_names = os.listdir(path) # contains all filenames within the selected folder
month_combis = [['02','03','04','05','06'], ['06','07','08','09','10'], ['10', '11', '12', '01', '02']]
seasons = ['summer', 'kharif', 'rabi']

for year in range(2016,2019):
    for m in month_combis:
        if m == month_combis[-1]:
            temp1 = [str(year) + x for x in m[:-2]]
            temp2 = [str(year+1) + x for x in m[3:]]
            year_month = temp1 + temp2
        else:
            year_month = [str(year) + x for x in m]

        group = []
        for k in range(len(m)):
            group = group + list(filter(lambda x: year_month[k] in x, file_names))

        n = len(group) # number of images within folder
        if n != 0: 
            images = []
            val_array = []
            bin_array = []

            for j in range(n):
                images.append(gdal.Open(path + group[j])) # Open each of the n images
                val_array.append(images[j].GetRasterBand(1).ReadAsArray().flatten())  # Read each of the n images as an array
                
            col = images[0].RasterXSize # number of columns
            rows = images[0].RasterYSize # number of rows
            driver = images[0].GetDriver()
            nodata_value = images[0].GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
            
            for k in range(n):
                bin_array.append(np.where(val_array[k] == nodata_value, 1, 0)) # For each image, create binary array where value is 1 if nodata pixel, 0 otherwise

            sum_counts = sum(bin_array) # For each pixel, we count the number of nodata values
            numbers = n - sum_counts # number of times we have data values per pixel
            no_data = np.where(sum_counts == n, 1, 0) # 1 if no data available for particular pixel, 0 otherwise

            # If numbers is 0 for a particular pixel, then we do not have any data on that pixel
            numbers = np.where(numbers == 0, 1, numbers) # Set to 1 as we can not divide by 0

            val_sum = sum(val_array) # Sum all values
            avg = (sum(val_array) - sum_counts * nodata_value + no_data * nodata_value) / numbers # Calculate the average-data
            avgdataMatrix= avg.reshape(rows,col)

            # Create a new raster to save the average-values
            raster_avg = driver.Create('LAI_' + str(year) + '_' + seasons[month_combis.index(m)] + ".tiff", col, rows, 1, gdal.GDT_Float32)

            # Copy the properties of the original raster
            raster_avg.SetGeoTransform(images[0].GetGeoTransform())
            raster_avg.SetProjection(images[0].GetProjection())

            # Add the average values to newly created raster
            raster_avg.GetRasterBand(1).WriteArray(avgdataMatrix)
            raster_avg.GetRasterBand(1).SetNoDataValue(nodata_value)

            #close the raster in order yo open in Qgis o Arcgis without problems.
            raster_avg = None
            del raster_avg

In [5]:
# AGGREGATION PER SOWING/HARVEST PERIOD

os.chdir('C:/Users/mieke/Documents/Msc Thesis/random_forest/preprocess/LAI/sow_harvest') # Set path
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/Copernicus/LAI/Karnataka/aoi_rescaled/' # Set path
file_names = os.listdir(path) # contains all filenames within the selected folder
month_combis = [['01','02'], ['02','03'], ['05', '06'], ['06', '07'], ['09', '10'], ['10', '11'], ['04'], ['08'], ['12']]
periods = ['rabi_harvest', 'summer_sow', 'summer_harvest', 'kharif_sow', 'kharif_harvest', 'rabi_sow', 'summer_between', 'kharif_between', 'rabi_between']

for year in range(2016,2019):
    for m in month_combis:
        if m == month_combis[0]:
            year = year + 1

        year_month = [str(year) + x for x in m]

        group = []
        for k in range(len(m)):
            group = group + list(filter(lambda x: year_month[k] in x, file_names))

        n = len(group) # number of images within folder
        if n != 0: 
            images = []
            val_array = []
            bin_array = []

            for j in range(n):
                images.append(gdal.Open(path + group[j])) # Open each of the n images
                val_array.append(images[j].GetRasterBand(1).ReadAsArray().flatten())  # Read each of the n images as an array
                
            col = images[0].RasterXSize # number of columns
            rows = images[0].RasterYSize # number of rows
            driver = images[0].GetDriver()
            nodata_value = images[0].GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
            
            for k in range(n):
                bin_array.append(np.where(val_array[k] == nodata_value, 1, 0)) # For each image, create binary array where value is 1 if nodata pixel, 0 otherwise

            sum_counts = sum(bin_array) # For each pixel, we count the number of nodata values
            numbers = n - sum_counts # number of times we have data values per pixel
            no_data = np.where(sum_counts == n, 1, 0) # 1 if no data available for particular pixel, 0 otherwise

            # If numbers is 0 for a particular pixel, then we do not have any data on that pixel
            numbers = np.where(numbers == 0, 1, numbers) # Set to 1 as we can not divide by 0

            val_sum = sum(val_array) # Sum all values
            avg = (sum(val_array) - sum_counts * nodata_value + no_data * nodata_value) / numbers # Calculate the average-data
            avgdataMatrix= avg.reshape(rows,col)

            if m == month_combis[0]:
                year = year - 1

            # Create a new raster to save the average-values
            raster_avg = driver.Create('LAI_' + str(year) + '_' + periods[month_combis.index(m)] + ".tiff", col, rows, 1, gdal.GDT_Float32)

            # Copy the properties of the original raster
            raster_avg.SetGeoTransform(images[0].GetGeoTransform())
            raster_avg.SetProjection(images[0].GetProjection())

            # Add the average values to newly created raster
            raster_avg.GetRasterBand(1).WriteArray(avgdataMatrix)
            raster_avg.GetRasterBand(1).SetNoDataValue(nodata_value)

            #close the raster in order yo open in Qgis o Arcgis without problems.
            raster_avg = None
            del raster_avg

LST

In [6]:
# AGGREGATION PER SEASON

os.chdir('C:/Users/mieke/Documents/Msc Thesis/random_forest/preprocess/LST/season') # Set path
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/GEE/LST/Karnataka/aoi_rescaled/' # Set path
file_names = os.listdir(path) # contains all filenames within the selected folder
month_combis = [['02','03','04','05','06'], ['06','07','08','09','10'], ['10', '11', '12', '01', '02']]
seasons = ['summer', 'kharif', 'rabi']

for year in range(2016,2019):
    for m in month_combis:
        if m == month_combis[-1]:
            temp1 = [str(year) + '_' + x for x in m[:-2]]
            temp2 = [str(year+1) + '_' + x for x in m[3:]]
            year_month = temp1 + temp2
        else:
            year_month = [str(year) + '_' + x for x in m]

        group = []
        for k in range(len(m)):
            group = group + list(filter(lambda x: year_month[k] in x, file_names))

        n = len(group) # number of images within folder
        if n != 0: 
            images = []
            val_array = []
            bin_array = []

            for j in range(n):
                images.append(gdal.Open(path + group[j])) # Open each of the n images
                val_array.append(images[j].GetRasterBand(1).ReadAsArray().flatten())  # Read each of the n images as an array
                
            col = images[0].RasterXSize # number of columns
            rows = images[0].RasterYSize # number of rows
            driver = images[0].GetDriver()
            nodata_value = images[0].GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
            
            for k in range(n):
                bin_array.append(np.where(val_array[k] == nodata_value, 1, 0)) # For each image, create binary array where value is 1 if nodata pixel, 0 otherwise

            sum_counts = sum(bin_array) # For each pixel, we count the number of nodata values
            numbers = n - sum_counts # number of times we have data values per pixel
            no_data = np.where(sum_counts == n, 1, 0) # 1 if no data available for particular pixel, 0 otherwise

            # If numbers is 0 for a particular pixel, then we do not have any data on that pixel
            numbers = np.where(numbers == 0, 1, numbers) # Set to 1 as we can not divide by 0

            val_sum = sum(val_array) # Sum all values
            avg = (sum(val_array) - sum_counts * nodata_value + no_data * nodata_value) / numbers # Calculate the average-data
            avgdataMatrix= avg.reshape(rows,col)

            # Create a new raster to save the average-values
            raster_avg = driver.Create('LST_' + str(year) + '_' + seasons[month_combis.index(m)] + ".tiff", col, rows, 1, gdal.GDT_Float32)

            # Copy the properties of the original raster
            raster_avg.SetGeoTransform(images[0].GetGeoTransform())
            raster_avg.SetProjection(images[0].GetProjection())

            # Add the average values to newly created raster
            raster_avg.GetRasterBand(1).WriteArray(avgdataMatrix)
            raster_avg.GetRasterBand(1).SetNoDataValue(nodata_value)

            #close the raster in order yo open in Qgis o Arcgis without problems.
            raster_avg = None
            del raster_avg

In [7]:
# AGGREGATION PER SOWING/HARVEST PERIOD

os.chdir('C:/Users/mieke/Documents/Msc Thesis/random_forest/preprocess/LST/sow_harvest') # Set path
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/GEE/LST/Karnataka/aoi_rescaled/' # Set path
file_names = os.listdir(path) # contains all filenames within the selected folder
month_combis = [['01','02'], ['02','03'], ['05', '06'], ['06', '07'], ['09', '10'], ['10', '11'], ['04'], ['08'], ['12']]
periods = ['rabi_harvest', 'summer_sow', 'summer_harvest', 'kharif_sow', 'kharif_harvest', 'rabi_sow', 'summer_between', 'kharif_between', 'rabi_between']

for year in range(2016,2019):
    for m in month_combis:
        if m == month_combis[0]:
            year = year + 1

        year_month = [str(year) + '_' + x for x in m]

        group = []
        for k in range(len(m)):
            group = group + list(filter(lambda x: year_month[k] in x, file_names))

        n = len(group) # number of images within folder
        if n != 0: 
            images = []
            val_array = []
            bin_array = []

            for j in range(n):
                images.append(gdal.Open(path + group[j])) # Open each of the n images
                val_array.append(images[j].GetRasterBand(1).ReadAsArray().flatten())  # Read each of the n images as an array
                
            col = images[0].RasterXSize # number of columns
            rows = images[0].RasterYSize # number of rows
            driver = images[0].GetDriver()
            nodata_value = images[0].GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
            
            for k in range(n):
                bin_array.append(np.where(val_array[k] == nodata_value, 1, 0)) # For each image, create binary array where value is 1 if nodata pixel, 0 otherwise

            sum_counts = sum(bin_array) # For each pixel, we count the number of nodata values
            numbers = n - sum_counts # number of times we have data values per pixel
            no_data = np.where(sum_counts == n, 1, 0) # 1 if no data available for particular pixel, 0 otherwise

            # If numbers is 0 for a particular pixel, then we do not have any data on that pixel
            numbers = np.where(numbers == 0, 1, numbers) # Set to 1 as we can not divide by 0

            val_sum = sum(val_array) # Sum all values
            avg = (sum(val_array) - sum_counts * nodata_value + no_data * nodata_value) / numbers # Calculate the average-data
            avgdataMatrix= avg.reshape(rows,col)

            if m == month_combis[0]:
                year = year - 1

            # Create a new raster to save the average-values
            raster_avg = driver.Create('LST_' + str(year) + '_' + periods[month_combis.index(m)] + ".tiff", col, rows, 1, gdal.GDT_Float32)

            # Copy the properties of the original raster
            raster_avg.SetGeoTransform(images[0].GetGeoTransform())
            raster_avg.SetProjection(images[0].GetProjection())

            # Add the average values to newly created raster
            raster_avg.GetRasterBand(1).WriteArray(avgdataMatrix)
            raster_avg.GetRasterBand(1).SetNoDataValue(nodata_value)

            #close the raster in order yo open in Qgis o Arcgis without problems.
            raster_avg = None
            del raster_avg

SSM

In [8]:
# AGGREGATION PER SEASON

os.chdir('C:/Users/mieke/Documents/Msc Thesis/random_forest/preprocess/SSM/season') # Set path
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/GEE/SSM/Karnataka/aoi_rescaled/' # Set path
file_names = os.listdir(path) # contains all filenames within the selected folder
month_combis = [['02','03','04','05','06'], ['06','07','08','09','10'], ['10', '11', '12', '01', '02']]
seasons = ['summer', 'kharif', 'rabi']

for year in range(2016,2019):
    for m in month_combis:
        if m == month_combis[-1]:
            temp1 = [str(year) + '-' + x for x in m[:-2]]
            temp2 = [str(year+1) + '-' + x for x in m[3:]]
            year_month = temp1 + temp2
        else:
            year_month = [str(year) + '-' + x for x in m]

        group = []
        for k in range(len(m)):
            group = group + list(filter(lambda x: year_month[k] in x, file_names))

        n = len(group) # number of images within folder
        if n != 0: 
            images = []
            val_array = []
            bin_array = []

            for j in range(n):
                images.append(gdal.Open(path + group[j])) # Open each of the n images
                val_array.append(images[j].GetRasterBand(1).ReadAsArray().flatten())  # Read each of the n images as an array
                
            col = images[0].RasterXSize # number of columns
            rows = images[0].RasterYSize # number of rows
            driver = images[0].GetDriver()
            nodata_value = images[0].GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
            
            for k in range(n):
                bin_array.append(np.where(val_array[k] == nodata_value, 1, 0)) # For each image, create binary array where value is 1 if nodata pixel, 0 otherwise

            sum_counts = sum(bin_array) # For each pixel, we count the number of nodata values
            numbers = n - sum_counts # number of times we have data values per pixel
            no_data = np.where(sum_counts == n, 1, 0) # 1 if no data available for particular pixel, 0 otherwise

            # If numbers is 0 for a particular pixel, then we do not have any data on that pixel
            numbers = np.where(numbers == 0, 1, numbers) # Set to 1 as we can not divide by 0

            val_sum = sum(val_array) # Sum all values
            avg = (sum(val_array) - sum_counts * nodata_value + no_data * nodata_value) / numbers # Calculate the average-data
            avgdataMatrix= avg.reshape(rows,col)

            # Create a new raster to save the average-values
            raster_avg = driver.Create('SSM_' + str(year) + '_' + seasons[month_combis.index(m)] + ".tiff", col, rows, 1, gdal.GDT_Float32)

            # Copy the properties of the original raster
            raster_avg.SetGeoTransform(images[0].GetGeoTransform())
            raster_avg.SetProjection(images[0].GetProjection())

            # Add the average values to newly created raster
            raster_avg.GetRasterBand(1).WriteArray(avgdataMatrix)
            raster_avg.GetRasterBand(1).SetNoDataValue(nodata_value)

            #close the raster in order yo open in Qgis o Arcgis without problems.
            raster_avg = None
            del raster_avg

In [9]:
# AGGREGATION PER SOWING/HARVEST PERIOD

os.chdir('C:/Users/mieke/Documents/Msc Thesis/random_forest/preprocess/SSM/sow_harvest') # Set path
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/GEE/SSM/Karnataka/aoi_rescaled/' # Set path
file_names = os.listdir(path) # contains all filenames within the selected folder
month_combis = [['01','02'], ['02','03'], ['05', '06'], ['06', '07'], ['09', '10'], ['10', '11'], ['04'], ['08'], ['12']]
periods = ['rabi_harvest', 'summer_sow', 'summer_harvest', 'kharif_sow', 'kharif_harvest', 'rabi_sow', 'summer_between', 'kharif_between', 'rabi_between']

for year in range(2016,2019):
    for m in month_combis:
        if m == month_combis[0]:
            year = year + 1

        year_month = [str(year) + '-' + x for x in m]

        group = []
        for k in range(len(m)):
            group = group + list(filter(lambda x: year_month[k] in x, file_names))

        n = len(group) # number of images within folder
        if n != 0: 
            images = []
            val_array = []
            bin_array = []

            for j in range(n):
                images.append(gdal.Open(path + group[j])) # Open each of the n images
                val_array.append(images[j].GetRasterBand(1).ReadAsArray().flatten())  # Read each of the n images as an array
                
            col = images[0].RasterXSize # number of columns
            rows = images[0].RasterYSize # number of rows
            driver = images[0].GetDriver()
            nodata_value = images[0].GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
            
            for k in range(n):
                bin_array.append(np.where(val_array[k] == nodata_value, 1, 0)) # For each image, create binary array where value is 1 if nodata pixel, 0 otherwise

            sum_counts = sum(bin_array) # For each pixel, we count the number of nodata values
            numbers = n - sum_counts # number of times we have data values per pixel
            no_data = np.where(sum_counts == n, 1, 0) # 1 if no data available for particular pixel, 0 otherwise

            # If numbers is 0 for a particular pixel, then we do not have any data on that pixel
            numbers = np.where(numbers == 0, 1, numbers) # Set to 1 as we can not divide by 0

            val_sum = sum(val_array) # Sum all values
            avg = (sum(val_array) - sum_counts * nodata_value + no_data * nodata_value) / numbers # Calculate the average-data
            avgdataMatrix= avg.reshape(rows,col)

            if m == month_combis[0]:
                year = year - 1

            # Create a new raster to save the average-values
            raster_avg = driver.Create('SSM_' + str(year) + '_' + periods[month_combis.index(m)] + ".tiff", col, rows, 1, gdal.GDT_Float32)

            # Copy the properties of the original raster
            raster_avg.SetGeoTransform(images[0].GetGeoTransform())
            raster_avg.SetProjection(images[0].GetProjection())

            # Add the average values to newly created raster
            raster_avg.GetRasterBand(1).WriteArray(avgdataMatrix)
            raster_avg.GetRasterBand(1).SetNoDataValue(nodata_value)

            #close the raster in order yo open in Qgis o Arcgis without problems.
            raster_avg = None
            del raster_avg